# Datawiser API — Quickstart

This notebook walks through each endpoint of the `datawiserai` Python client.

**Prerequisites:**
```bash
pip install 'datawiserai[pandas]'
```

Set your API key as an environment variable (`DATAWISER_API_KEY`) or paste it below.

In [ ]:
import os
import datawiserai as dw

API_KEY = os.environ.get("DATAWISER_API_KEY", "your_api_key_here")
TICKER = "OLP"

client = dw.Client(api_key=API_KEY)
print(f"datawiserai v{dw.__version__}  —  connected")

## Universe

Discover which tickers are available for a given endpoint.

In [ ]:
u = client.universe("free-float")
print(f"Available tickers: {u.tickers}")
print(f"'{TICKER}' in universe: {TICKER in u}")
u.to_dataframe()

## Free Float

Time-series of free-float factor, percentage, shares outstanding, and excluded shares.

In [ ]:
ff = client.free_float(TICKER)
print(f"ticker={ff.ticker}  security_id={ff.security_id}  events={len(ff)}")

latest = ff.latest()
print(f"Latest: {latest.as_of}  factor={latest.free_float_factor:.4f}  pct={latest.free_float_pct:.2f}%")

ff.to_dataframe()

## Shares Outstanding

Share-count observations sourced from 10-K, 10-Q, and other filings.

In [ ]:
so = client.shares_outstanding(TICKER)
print(f"ticker={so.ticker}  events={len(so)}")

latest_so = so.latest()
print(f"Latest: {latest_so.as_of}  type={latest_so.share_type}  shares={latest_so.shares:,.0f}")

so.to_dataframe()

## Reference Data

Identifiers, company information, and security metadata.

In [ ]:
ref = client.reference(TICKER)
print(f"ticker={ref.ticker}  company={ref.company_name}")
print(f"cik={ref.cik}  lei={ref.lei}  mic={ref.mic}  is_primary={ref.is_primary}")
print(f"\nCompany info: {ref.company_info}")
print(f"\nSecurity info: {ref.security_info}")

## Free Float Events — High-Level Summary

One row per event date with aggregate metrics.  
`is_rebal` is **True** when the event is a rebalance (e.g. new **DEF 14A** proxy).

In [ ]:
ffe = client.free_float_events(TICKER)
df_events = ffe.to_event_summary_dataframe()
df_events.head(10)

## Free Float Events — Flat Owner Summary

One row per owner per event date (top-level component fields only).

In [ ]:
df_owners = ffe.to_dataframe()
print(f"owner-rows={len(df_owners)}  distinct dates={len(ffe.dates())}")
df_owners[["as_of", "name", "shares", "delta_shares", "entity_type", "is_officer"]].head(10)

## Free Float Events — Detail Drill-Down

Full nested ownership structure for interactive exploration.

In [ ]:
detail = client.free_float_events_detail(TICKER)
print(f"Event dates: {detail.dates[:5]}  ...\n")

ev = detail[0]
print(f"Event {ev.as_of} — {len(ev.owner_ids)} owners:")
for oid, name in list(ev.owner_names.items())[:5]:
    owner = ev.owner(oid)
    print(
        f"  {name:35s}  shares={owner['shares']:>12,.2f}  "
        f"components={len(owner.get('components', []))}  "
        f"restrictions={len(owner.get('restrictions', []))}"
    )